In [1]:
# read in data and packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from keras.utils import np_utils
#import bamboolib as bam

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import StandardScaler
from keras.utils import np_utils
from tensorflow import keras
from tensorflow.keras import layers
from keras_tuner.tuners import RandomSearch
import keras_tuner as kt

In [2]:
# read in data
df = pd.read_csv('CS_DATA_ADJ.csv')

In [3]:
#import bamboolib as bam

In [4]:
df1 = df.loc[((df['Year'] == 2019) | (df['Year'] == 2018) | (df['Year'] == 2017) | (df['Year'] == 2016) | (df['Year'] == 2015))]

In [5]:
df1 = df1.dropna(subset=['FWD_Return_3M_Class'])

In [6]:
# replace NA's with median 
df1 = df1.fillna(df1.median())

In [7]:
df1.describe()

,Mkt_Cap,Year,St_Sc_2021,St_Sc_2019,St_Sc_R_2021,St_Sc_R_2019,FWD_Return_136M_Class,FWD_Return_1M,FWD_Return_1M_REL,FWD_Return_1M_Class,...,FPE_6M_Rev,FPE_3M_Rev_REL,FPE_3M_Rev,FPE_1M_Rev_REL,FPE_1M_Rev,FPE_5Y_Pct_REL,FPE_5Y_Pct,FPE_vRIY_REL,FPE_vRIY,FPE
count,1.700460e+05,170046.000000,170046.000000,170046.000000,170046.000000,170046.000000,170046.000000,170046.000000,170046.000000,170046.000000,...,170046.000000,170046.000000,170046.000000,170046.000000,170046.000000,170046.000000,170046.000000,170046.000000,170046.000000,1.700460e+05
mean,3.174837e+04,2016.993261,50.058050,50.026887,50.054271,50.041124,0.262523,0.000340,50.049985,0.499618,...,-0.494463,50.047368,-0.495334,50.047453,-0.486659,49.678934,50.118429,49.949923,1.894969,2.916133e+01
std,7.118720e+04,1.416885,28.654393,28.644197,28.655435,28.676549,0.440007,0.071729,28.822902,0.500001,...,164.433726,28.227050,164.156935,28.273607,163.838563,28.266990,33.496271,28.308917,164.165455,2.487080e+03
min,7.357240e+01,2015.000000,0.200000,0.200000,0.200000,0.200000,0.000000,-0.831020,0.200000,0.000000,...,-67351.213680,0.200000,-67338.858690,0.200000,-67216.474470,0.100000,0.300000,0.100000,0.098379,1.590000e+00
25%,5.291727e+03,2016.000000,25.300000,25.300000,25.300000,25.300000,0.000000,-0.035577,25.000000,0.000000,...,-0.080675,26.200000,-0.060039,26.100000,-0.033531,25.700000,18.200000,25.900000,0.760915,1.174700e+01
50%,1.085680e+04,2017.000000,50.100000,50.000000,50.100000,50.000000,0.000000,0.000000,50.000000,0.000000,...,-0.001486,50.000000,-0.001971,50.000000,-0.000106,49.900000,49.800000,50.000000,1.000000,1.547500e+01
75%,2.680531e+04,2018.000000,74.800000,74.600000,74.800000,74.800000,1.000000,0.035302,75.000000,1.000000,...,0.078419,73.900000,0.056916,74.000000,0.032127,73.900000,82.500000,74.000000,1.307932,2.031500e+01
max,1.287650e+06,2019.000000,99.900000,99.900000,99.900000,99.900000,1.000000,1.849340,99.900000,1.000000,...,3241.804898,99.900000,1852.023819,99.900000,1811.120531,99.800000,100.000000,99.800000,67368.629520,1.020500e+06


### FWD_Return_3M_Class

In [8]:
nn = df1.drop(columns=['FWD_Return_136M_Class',
                         'FWD_Return_1M_REL', 'FWD_Return_1M', 'FWD_Return_1M_Class',
                         'FWD_Return_3M_REL', 'FWD_Return_3M', 
                         'FWD_Return_6M_Class', 'FWD_Return_6M', 'FWD_Return_6M_REL',
                         'Date', 'Year', 'ticker', 'Mkt_Cap', 'Sub_Industry', 'Sector', 
                         'St_Sc_2021', 'St_Sc_2019', 'St_Sc_R_2021', 'St_Sc_R_2019',  
                         'FWD_EPS_3M','FWD_EPS_3M_REL', 'FWD_EPS_3M_REL_Decile',  'FWD_EPS_GR_3M_REV', 'FWD_EPS_GR_3M_REV_REL', 'FWD_EPS_GR_3M_REV_REL_Decile',
                         'FWD_REV_3M','FWD_REV_3M_REL', 'FWD_REV_3M_REL_Decile',  'FWD_REV_GR_3M_REV', 'FWD_REV_GR_3M_REV_REL', 'FWD_REV_GR_3M_REV_REL_Decile',
                         'FWD_FPE_3M_Rev_REL','FWD_FPE_3M_Rev', 'FWD_FPE_3M_Rev_REL_Decile', 
                         'FWD_DY_3M_Rev_REL', 'FWD_DY_3M_Rev', 'FWD_DY_3M_Rev_REL_Decile',
                         'FWD_EVS_3M_Rev_REL', 'FWD_EVS_3M_Rev', 'FWD_EVS_3M_Rev_REL_Decile',
                         'FWD_PB_3M_Rev_REL', 'FWD_PB_3M_Rev', 'FWD_PB_3M_Rev_REL_Decile'])

In [9]:
# separate data into X and y
X = nn.drop(columns=['FWD_Return_3M_Class',])
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 170046 entries, 84760 to 254931
Columns: 370 entries, An_Mark_6M_REL to FPE
dtypes: float64(364), int64(6)
memory usage: 481.3 MB


In [10]:
# y = response variable 
y = nn['FWD_Return_3M_Class']

In [11]:
# load packages
from sklearn.preprocessing import StandardScaler
# explore different scalers 
ss = StandardScaler()
# fit explanatory variables with each scaler method
ss.fit(X)
# transform explanatory variables
Xs = ss.transform(X)

In [12]:
from sklearn.model_selection import train_test_split
# Split the dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(Xs, y, test_size = 0.2, random_state = 2)

In [13]:
X_train.shape

(136036, 370)

In [45]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + 
    K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [49]:
from tensorflow.keras.callbacks import EarlyStopping
safety = EarlyStopping(monitor='val_loss', patience=5, min_delta=0)
eval = 'acc'

In [50]:
# chosen parameters after tuning
activation = 'tanh'
units = 200

In [51]:
tb_model = tf.keras.Sequential()
tb_model.add(tf.keras.Input(shape=(370,)))
tb_model.add(tf.keras.layers.Dense(units, activation = activation))
tb_model.add(tf.keras.layers.Dense(units, activation = activation))
tb_model.add(tf.keras.layers.Dense(units, activation = activation))
tb_model.add(tf.keras.layers.Dense(units, activation = activation))
tb_model.add(tf.keras.layers.Dense(1, activation = 'sigmoid')) 
tb_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=eval)

In [52]:
%%time
tb_model.fit(X_train, y_train, epochs = 1000, validation_data=(X_test, y_test), batch_size = 20, callbacks=[safety])

Epoch 1/1000
6802/6802 [==============================] - 25s 4ms/step - loss: 0.6926 - acc: 0.5436 - val_loss: 0.6895 - val_acc: 0.5566
Epoch 2/1000
6802/6802 [==============================] - 25s 4ms/step - loss: 0.6699 - acc: 0.5923 - val_loss: 0.6539 - val_acc: 0.6198
Epoch 3/1000
6802/6802 [==============================] - 22s 3ms/step - loss: 0.6242 - acc: 0.6576 - val_loss: 0.6132 - val_acc: 0.6665
Epoch 4/1000
6802/6802 [==============================] - 23s 3ms/step - loss: 0.5639 - acc: 0.7147 - val_loss: 0.5839 - val_acc: 0.6986
Epoch 5/1000
6802/6802 [==============================] - 23s 3ms/step - loss: 0.5104 - acc: 0.7565 - val_loss: 0.5425 - val_acc: 0.7321
Epoch 6/1000
6802/6802 [==============================] - 25s 4ms/step - loss: 0.4667 - acc: 0.7852 - val_loss: 0.5252 - val_acc: 0.7455
Epoch 7/1000
6802/6802 [==============================] - 26s 4ms/step - loss: 0.4321 - acc: 0.8066 - val_loss: 0.5176 - val_acc: 0.7539
Epoch 8/1000
6802/6802 [=================

In [54]:
# evaluate the keras model
_, ACC = tb_model.evaluate(X_test, y_test)
nn_acc=print('Accuracy: %.2f' % (ACC*100))
nn_acc

1063/1063 [==============================] - 3s 2ms/step - loss: 0.4886 - acc: 0.7966
Accuracy: 79.66
